## Validation on titanic dataset

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/fuyu-quant/CreateTool-AGI/blob/main/examples/titanic.ipynb)

In [1]:
%%capture
!pip install git+https://github.com/fuyu-quant/CreateTool-AGI.git

In [2]:
from createtoolagi import CTAGI

#import os
#os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

import pkg_resources
package_name = "createtoolagi"  
pkg = pkg_resources.get_distribution(package_name)
print(pkg.version)

1.0.3


## Prepare training data

In [4]:
import csv

training_data = ''
with open('../data/learning.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        training_data += ','.join(row) + '\n'


In [5]:
predict_data = "2,male,27.0,0,0,13.0,S,Second,man,True,,Southampton,no,True"

In [6]:
input = """
{training_data}_
------------------
Answer between 0~1 what happened to the "SURVIVED" of the following people when the above data was available.
Output should be numeric only.
------------------
pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
{predict_data_}
""".format(training_data = training_data, predict_data_ = predict_data)

## Normal GPT-4

In [21]:
from langchain.llms import OpenAI

gpt4 = OpenAI(temperature=0, model_name = 'gpt-4')

In [22]:
gpt4(input)

'0'

## CreateTool-AGI

In [11]:
!rm -r ../tools/titanic

In [12]:
from createtoolagi import CTAGI

from qdrant_client import QdrantClient
from qdrant_client import models

base_model_name = 'gpt-4'
#base_model_name = 'gpt-3.5-turbo'
create_model_name = 'gpt-4'
embegging_model_name = 'text-embedding-ada-002'

qdrant = QdrantClient(path='../tools/titanic')
qdrant.create_collection(collection_name ="tool_store", vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE))

ctagi = CTAGI(
    base_model_name=base_model_name, 
    create_model_name=create_model_name, 
    embegging_model_name=embegging_model_name,
    qdrant=qdrant
    )

In [13]:

ctagi.run(input=input)

> Search tools.
Search results：
[]

> Decide if the task is executable.
Response：No.
I must create the tool before executing.

Try:1
> Create a tool.
Completed!
Created tool name：SurvivalPredictionTool

Created tool code: from langchain.tools import BaseTool

class SurvivalPredictionTool(BaseTool):
    name = "SurvivalPredictionTool"
    description = "used for predicting survival based on input data. The input is a single string with comma-separated values."

    def _run(self, query: str) -> str:
        "Use the tool."
        data = query.split(",")

        pclass, sex, age, sibsp, parch, fare, embarked, _class, who, adult_male, deck, embark_town, alive, alone = data

        # Basic logic for survival prediction
        if sex == "female":
            if pclass == "1" or pclass == "2":
                return "1"
            elif age <= "18":
                return "1"
            else:
                return "0"
        else:
            if pclass == "1" and age <= "18":
        

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/createtoolagi/ctagi.py", line 72, in run
    self.executor.run_with_create_tool(input, tool_code = tool_code)
  File "/opt/conda/lib/python3.10/site-packages/createtoolagi/executor.py", line 62, in run_with_create_tool
    agent.run(input)
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 236, in run
    return self(args[0], callbacks=callbacks)[self.output_keys[0]]
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 140, in __call__
    raise e
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 134, in __call__
    self._call(inputs, run_manager=run_manager)
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 905, in _call
    next_step_output = self._take_next_step(
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 749, in _take_next_step
    raise e
  File 

Completed!
Created tool name：SurvivalPredictionTool

Created tool code: from langchain.tools import BaseTool

class SurvivalPredictionTool(BaseTool):
    name = "SurvivalPredictionTool"
    description = "used for predicting survival based on input data. The input is a single string with comma-separated values."

    def _run(self, query: str) -> str:
        "Use the tool."
        data = query.split(",")

        pclass, sex, age, sibsp, parch, fare, embarked, _class, who, adult_male, deck, embark_town, alive, alone = data

        # Convert age and fare to float
        age = float(age) if age else None
        fare = float(fare) if fare else None

        # Basic logic for survival prediction
        if sex == "female":
            if pclass == "1" or pclass == "2":
                return "1"
            elif age and age <= 18:
                return "1"
            else:
                return "0"
        else:
            if pclass == "1" and age and age <= 18:
                re

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/createtoolagi/ctagi.py", line 72, in run
    self.executor.run_with_create_tool(input, tool_code = tool_code)
  File "/opt/conda/lib/python3.10/site-packages/createtoolagi/executor.py", line 62, in run_with_create_tool
    agent.run(input)
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 236, in run
    return self(args[0], callbacks=callbacks)[self.output_keys[0]]
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 140, in __call__
    raise e
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 134, in __call__
    self._call(inputs, run_manager=run_manager)
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 905, in _call
    next_step_output = self._take_next_step(
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 749, in _take_next_step
    raise e
  File 

Completed!
Created tool name：SurvivalPredictionTool

Created tool code: from langchain.tools import BaseTool

class SurvivalPredictionTool(BaseTool):
    name = "SurvivalPredictionTool"
    description = "used for predicting survival based on input data. The input is a single string with comma-separated values."

    def _run(self, query: str) -> str:
        "Use the tool."
        data = query.split(",")

        pclass, sex, age, sibsp, parch, fare, embarked, _class, who, adult_male, deck, embark_town, alive, alone = data

        # Convert age and fare to float
        age = float(age) if age else None
        fare = float(fare) if fare else None

        # Basic logic for survival prediction
        if sex == "female":
            if pclass == "1" or pclass == "2":
                return "1"
            elif age and age <= 18:
                return "1"
            else:
                return "0"
        else:
            if pclass == "1" and age and age <= 18:
                re

...

Completed!
Created tool name：SurvivalPredictionTool

Created tool code: from langchain.tools import BaseTool

class SurvivalPredictionTool(BaseTool):
    name = "SurvivalPredictionTool"
    description = "used for predicting survival based on input data. The input is a single string with comma-separated values."

    def _run(self, query: str) -> str:
        "Use the tool."
        data = query.split(",")

        pclass, sex, age, sibsp, parch, fare, embarked, _class, who, adult_male, deck, embark_town, alive, alone = data

        # Convert age and fare to float
        age = float(age) if age else None
        fare = float(fare) if fare else None

        # Basic logic for survival prediction
        if sex == "female":
            if pclass == "1" or pclass == "2":
                return "1"
            elif age and age <= 18:
                return "1"
            else:
                return "0"
        else:
            if pclass == "1" and age and age <= 18:
                re

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/createtoolagi/ctagi.py", line 72, in run
    self.executor.run_with_create_tool(input, tool_code = tool_code)
  File "/opt/conda/lib/python3.10/site-packages/createtoolagi/executor.py", line 62, in run_with_create_tool
    agent.run(input)
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 236, in run
    return self(args[0], callbacks=callbacks)[self.output_keys[0]]
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 140, in __call__
    raise e
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 134, in __call__
    self._call(inputs, run_manager=run_manager)
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 905, in _call
    next_step_output = self._take_next_step(
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 749, in _take_next_step
    raise e
  File 

Completed!
Created tool name：SurvivalPredictionTool

Created tool code: from langchain.tools import BaseTool

class SurvivalPredictionTool(BaseTool):
    name = "SurvivalPredictionTool"
    description = "used for predicting survival based on input data. The input is a single string with comma-separated values."

    def _run(self, query: str) -> str:
        "Use the tool."
        data = query.split(",")

        pclass, sex, age, sibsp, parch, fare, embarked, _class, who, adult_male, deck, embark_town, alive, alone = data

        # Convert age and fare to float
        age = float(age) if age else None
        fare = float(fare) if fare else None

        # Basic logic for survival prediction
        if sex == "female":
            if pclass == "1" or pclass == "2":
                return "1"
            elif age and age <= 18:
                return "1"
            else:
                return "0"
        else:
            if pclass == "1" and age and age <= 18:
                re

Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/createtoolagi/ctagi.py", line 72, in run
    self.executor.run_with_create_tool(input, tool_code = tool_code)
  File "/opt/conda/lib/python3.10/site-packages/createtoolagi/executor.py", line 62, in run_with_create_tool
    agent.run(input)
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 236, in run
    return self(args[0], callbacks=callbacks)[self.output_keys[0]]
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 140, in __call__
    raise e
  File "/opt/conda/lib/python3.10/site-packages/langchain/chains/base.py", line 134, in __call__
    self._call(inputs, run_manager=run_manager)
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 905, in _call
    next_step_output = self._take_next_step(
  File "/opt/conda/lib/python3.10/site-packages/langchain/agents/agent.py", line 749, in _take_next_step
    raise e
  File 

KeyboardInterrupt: 

## Predict

In [ ]:
pred_data = "3,male,32.0,0,0,7.75,Q,Third,man,True,,Queenstown,no,True"

input = """
Please predict the following titanic passengers.
------------------
pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
{pred_data_}
""".format(pred_data_ = pred_data)


In [ ]:
ctagi.run(input=input)